# Problem statement

Q: How much does it cost to cool a skyscraper in the summer?
A: A lot! And not just in dollars, but in environmental impact.

Thankfully, significant investments are being made to improve building efficiencies to reduce costs and emissions. The question is, are the improvements working? That’s where you come in. Under pay-for-performance financing, the building owner makes payments based on the difference between their real energy consumption and what they would have used without any retrofits. The latter values have to come from a model. Current methods of estimation are fragmented and do not scale well. Some assume a specific meter type or don’t work with different building types.

In this competition, you’ll develop accurate models of metered building energy usage in the following areas: chilled water, electric, hot water, and steam meters. The data comes from over 1,000 buildings over a three-year timeframe. With better estimates of these energy-saving investments, large scale investors and financial institutions will be more inclined to invest in this area to enable progress in building efficiencies.

### Note about codestyle

In Jupyter notebooks, the emphasis is on quick experimentation. The quality of the code is not what we optimize for. So when "productionizing" this notebook, take everything with a grain of salt and rethink the structure of the code.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold

sns.set_style('darkgrid')

In [ ]:
%matplotlib inline

Input data is available on the file system in `../input/ashrae-energy-prediction`. Let's just list it first.

In [ ]:
DATA_DIR = '../input/ashrae-energy-prediction'
print(os.listdir(DATA_DIR))

# Description of files

(pasted from https://www.kaggle.com/c/ashrae-energy-prediction/data)

### train.csv
* `building_id` - Foreign key for the building metadata.
* `meter` - The meter id code. Read as `{0: electricity, 1: chilledwater, 2: steam, 3: hotwater}`. Not every building has all meter types.
* `timestamp` - When the measurement was taken
* `meter_reading` - The target variable. Energy consumption in kWh (or equivalent). Note that this is real data with measurement error, which we expect will impose a baseline level of modeling error. UPDATE: as discussed here, the site 0 electric meter readings are in kBTU.

### building_meta.csv
* `site_id` - Foreign key for the weather files.
* `building_id` - Foreign key for training.csv
* `primary_use` - Indicator of the primary category of activities for the building based on EnergyStar property type definitions
* `square_feet` - Gross floor area of the building
* `year_built` - Year building was opened
* `floor_count` - Number of floors of the building

### weather_[train/test].csv
Weather data from a meteorological station as close as possible to the site.

* `site_id`
* `air_temperature` - Degrees Celsius
* `cloud_coverage` - Portion of the sky covered in clouds, in oktas
* `dew_temperature` - Degrees Celsius
* `precip_depth_1_hr` - Millimeters
* `sea_level_pressure` - Millibar/hectopascals
* `wind_direction` - Compass direction (0-360)
* `wind_speed` - Meters per second

### test.csv
The submission files use row numbers for ID codes in order to save space on the file uploads. `test.csv` has no feature data; it exists so you can get your predictions into the correct order.

* `row_id` - Row id for your submission file
* `building_id` - Building id code
* `meter` - The meter id code
* `timestamp` - Timestamps for the test data period

# Load up the data

In [ ]:
# Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def load_df(fname):
    df = pd.read_csv(os.path.join(DATA_DIR, fname))
    if 'timestamp' in df.columns:
        # I guess fortunately all timestamp columns are called `timestamp`.
        df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

In [ ]:
train_df = reduce_mem_usage(load_df('train.csv'))
building_metadata_df = reduce_mem_usage(load_df('building_metadata.csv'))
weather_train_df = reduce_mem_usage(load_df('weather_train.csv'))

# Exploratory data analysis, minor cleanup and feature generation

In [ ]:
def describe_df(df):
    print('Shape of data: ', df.shape)
    print('\nBasic info:')
    print(df.info())
    print('\nQuick peek at the data:')
    print(df.head())
    print('\nBasic description of the data:')
    print(df.describe())
    print('\nLooking at NAs')
    print(df.isna().sum())

## Analysis: `train_df`

In [ ]:
describe_df(train_df)

There are no NAs in `train_df`, so that's nice. The timestamps have been treated well. The memory consumption is modest, so there's no need to mess around with that.

It's worth looking at the meter readings as timeseries data.

In [ ]:
# The duration of the training data
train_df['timestamp'].min(), train_df['timestamp'].max()

In [ ]:
sns.relplot(x='timestamp', y='meter_reading', hue='meter', kind='line', 
            palette=sns.color_palette('hls', 4), aspect=16/9,
            data=(train_df[train_df['timestamp'] > pd.to_datetime('2016-12-01')]
                  .groupby(by=['meter', 'timestamp'])
                  .agg({'meter_reading': 'median'}).reset_index()))
plt.xticks(rotation=15)
plt.title('Median meter readings over time for different meter types (1 month)')
plt.gca().set(yscale='log')
plt.ylabel('Meter reading (log-scale)')
plt.show()

> Note: The `meter_reading` column is in log-scale above. The outputs of meter type `2` are much higher than the rest, so it's easier to see patterns in log-scale.

In the plot above, we can see that the meter type is obviously an important feature. In addition, clearly the `meter_reading` has plenty of seasonality. The most evident examples of seasonality here are based on time of day and day of week and that seasonality is different for different meter types.

For example, notice that meter `0` is clearly affected by weekend vs. weekday dynamics more so than other meter types. There are probably also monthly seasonal effects based on heating needs being different in winter vs. summer, although that's not visible in the graph above. To see that aspect of seasonality look at the plot below.

In [ ]:
sns.relplot(x='month_of_year', y='meter_reading', hue='meter', kind='line', 
            palette=sns.color_palette('hls', 4), aspect=16/9,
            data=(train_df
                  .assign(month_of_year=train_df['timestamp'].dt.month)
                  .groupby(by=['meter', 'month_of_year'])
                  .agg({'meter_reading': 'median'}).reset_index()))
plt.xticks(rotation=15)
plt.title('Median meter readings over months for different meter types (1 year)')
plt.show()
plt.close()

So far, we've concluded that the meter type, the hour of day, the day of week, the day of year are potentially useful features. So let's add those to the `train_df` dataframe.

## Feature generation: `train_df`

In [ ]:
train_df = train_df.assign(hour_of_day=train_df['timestamp'].dt.hour, 
                           day_of_week=train_df['timestamp'].dt.dayofweek,
                           day_of_year=train_df['timestamp'].dt.dayofyear)

## Analysis: `building_metadata.csv`

The `train_df` dataframe had a `building_id` column that we never investigated. Let's do that now, combined with the `building_metadata.csv` file.

In [ ]:
describe_df(building_metadata_df)

One of the problems here is that the columns `year_built` and `floor_count` have plenty of null values. For now, I'm planning to leave them as is. Lightgbm can handle null values, so we'll rely on that for now. 

It would be interesting to see how both `square_feet`, `primary_use` and `year_built` influence `energy_consumption`

### Relationship between `square_feet` and energy consumption

In [ ]:
g = sns.jointplot(x='square_feet', y='meter_reading', height=8,
                  data=(train_df.groupby(by='building_id')
                        .agg({'meter_reading': 'median'})
                        .join(building_metadata_df, on=['building_id'])))
plt.show()
plt.close()

It's interesting that there doesn't seem to be much of a linear relationship between `square_feet` and the `meter_reading`. What's more interesting is that the relationship is more evident in log-log space.

In [ ]:
g = sns.jointplot(x='square_feet', y='meter_reading', height=8, kind='reg',
                  data=(train_df.groupby(by='building_id')
                        .agg({'meter_reading': 'median'})
                        .join(building_metadata_df, on=['building_id'])
                        .pipe(lambda df: df.assign(meter_reading=np.log1p(df.meter_reading),
                                                   square_feet=np.log1p(df.square_feet)))))
g.ax_joint.set_xlabel('Meter reading (log-scale)')
g.ax_joint.set_ylabel('Square feet (log-scale)')
g.fig.suptitle('Relationship between Square footage and energy consumption in log-log space')
plt.show()
plt.close()

### Relationship between `primary_use` and energy consumption

In [ ]:
fig = plt.figure(figsize=(10, 20))
sns.violinplot(x='meter_reading', y='primary_use', orient='h', scale='count',
               data=(train_df.groupby(by='building_id')
                     .agg({'meter_reading': 'median'})
                     .join(building_metadata_df, on=['building_id'])))
plt.show()
plt.close()

As could have been expected, the distributions of energy consumption for various types of buildings is different. Since the violin plots are scaled by the count of datapoints, it means that most of the data in the training set is for education buildings.

### Relationship between `year_built` and energy consumption

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(x='year_built', y='meter_reading',
             data=(train_df
                   .merge(building_metadata_df, on=['building_id'])
                   .groupby(by='year_built')
                   .agg({'meter_reading': 'median'})
                   .reset_index()))
plt.title('Meter readings for buildings built in different years')
plt.show()
plt.close()

Clearly, the median meter reading for buildings built during different years is very different. So of course the year_built is an important feature. Perhaps it would be appropriate to keep it a categorical feature. 

## Feature generation: `building_metadata.csv`

For now, we definitely want to encode the 2 categorical variables: `year_built` and `primary_use`. We also want to log-transform the `square_feet` column because of our previous analysis.

In [ ]:
building_metadata_enc = {
    'year_built': LabelEncoder(),
    'primary_use': LabelEncoder(),
}
building_metadata_df['year_built_enc'] = (building_metadata_enc['year_built']
                                          .fit_transform(building_metadata_df['year_built']))
building_metadata_df['primary_use_enc'] = (building_metadata_enc['primary_use']
                                           .fit_transform(building_metadata_df['primary_use']))

In [ ]:
building_metadata_df['square_feet_log'] = np.log1p(building_metadata_df['square_feet'])

## Analysis: `weather_train.csv`

In [ ]:
describe_df(weather_train_df)

There are lot of nans here. We might have to do some imputation for weather. Plus, we don't even know if all the hours starting from the smallest in the dataframe to the largest have values here. We should check that out first.

### Checking time-gaps in weather data

In [ ]:
training_datetime_range = pd.date_range(start=train_df['timestamp'].min(), end=train_df['timestamp'].max(), freq='H')
sites = building_metadata_df['site_id'].unique()
weather_train_idx = pd.MultiIndex.from_product([training_datetime_range, sites], names=['timestamp', 'site_id'])

weather_train_idx

As you can see, the `weather_train_idx` index has a length larger than that of `weather_train_df`. This implies that there are gaps in the weather information. Let's expand the `weather_train_df` dataframe to account for those gaps first.

In [ ]:
weather_train_df = pd.merge(left=pd.DataFrame(index=weather_train_idx).reset_index(), 
                            right=weather_train_df, how='left', on=['timestamp', 'site_id'])

In [ ]:
weather_train_df

### Imputing weather information

We can use `interpolate` to impute values. We do so within the context of each `site_id`.

In [ ]:
weather_train_df = pd.concat([site_weather_train_df.sort_values('timestamp').interpolate(limit_direction='both')
                             for _, site_weather_train_df in weather_train_df.groupby('site_id')])

In [ ]:
weather_train_df

In [ ]:
describe_df(weather_train_df)

Many of the columns still have plenty of NAs. The only explanation for it is that this must be for sites that have no data for those columns. Let's just double check that.

In [ ]:
weather_train_df[weather_train_df['cloud_coverage'].isna()]['site_id'].unique()

In [ ]:
print(weather_train_df[weather_train_df['site_id'] == 7]['cloud_coverage'].shape)
print(weather_train_df[weather_train_df['site_id'] == 7]['cloud_coverage'].isna().sum())
print(weather_train_df[weather_train_df['site_id'] == 11]['cloud_coverage'].shape)
print(weather_train_df[weather_train_df['site_id'] == 11]['cloud_coverage'].isna().sum())

It's no wonder that all the number of NAs in the post-interpolation `weather_train_df` are multiples of `8784`. We'll just not use these columns for now in the training data.

# Merge all the dataframes

Before we do the analysis of the relationship of the various weather parameters to the meter reading, it might be beneficial to merge the dataframes, so we can get ahead of this expensive operation.

In [ ]:
merged_df = (train_df
             .merge(building_metadata_df, on=['building_id'])
             .merge(weather_train_df, on=['site_id', 'timestamp']))

In [ ]:
merged_df.head()

### Relationship between `air_temperature` and `meter_reading`

In [ ]:
sns.jointplot(data=merged_df.assign(meter_reading=np.log1p(merged_df.meter_reading)), kind='hex',
              x='air_temperature', y='meter_reading')
plt.show()
plt.close()

In [ ]:
for primary_use, group_df in merged_df.groupby('primary_use'):
    sns.jointplot(data=group_df.assign(meter_reading=np.log1p(group_df.meter_reading)), kind='hex', 
                  x='air_temperature', y='meter_reading')
    plt.title(primary_use)
    plt.show()
    plt.close()

Presumably there are plenty of moments where the various buildings are just not in use and it doesn't matter how hot or cold it is outside. So we should ignore the low `meter_reading` data points. Also, we'll stick to using log-space for meter readings because we've decided that the strong correlation between `square_feet` and `meter_reading` in log-log space is useful for us.

It doesn't look like there's a very clear relationship between the 2 variables. The shape is mostly a blob, but there are density differences in the blob. When split across primary_use, we start to see some patterns for some of the primary usages, but nothing that's very clear. So might be okay to keep the variable in, but I'm not expecting much from it.

### Relationship between `dew_temperature` and `meter_reading`

In [ ]:
sns.jointplot(data=merged_df.assign(meter_reading=np.log1p(merged_df.meter_reading)), kind='hex',
              x='dew_temperature', y='meter_reading')
plt.show()
plt.close()

In [ ]:
for primary_use, group_df in merged_df.groupby('primary_use'):
    sns.jointplot(data=group_df.assign(meter_reading=np.log1p(group_df.meter_reading)), kind='hex', 
                  x='dew_temperature', y='meter_reading')
    plt.title(primary_use)
    plt.show()
    plt.close()

This whole thing also looks very similar to `air_temperature`. Perhaps worth keeping this in as well, but low expectations.

# Prepare training data

We established earlier that there is value in dealing with the meter_reading in log-space. So we'll do that transformation here as well.

In [ ]:
merged_df['meter_reading_log'] = np.log1p(merged_df['meter_reading'])

Of course there are plenty of columns in the `merged_train_df` dataframe that are not required for the actual modeling. So let's select the columns we care for. Let's also make a separate dataframe for testing.

In [ ]:
print(sorted(merged_df.columns))

In [ ]:
merged_df = merged_df.sort_values('timestamp')
feature_cols = ['building_id', 'day_of_week', 'day_of_year', 'floor_count', 
                'hour_of_day', 'meter', 'primary_use_enc', 'site_id', 
                'square_feet_log', 'year_built_enc', 'air_temperature', 'dew_temperature']
categorical_features = ['building_id', 'day_of_week', 'day_of_year', 'hour_of_day',
                        'meter', 'primary_use_enc', 'site_id', 'year_built_enc']
X_df = merged_df[feature_cols]
y_df = merged_df[['meter_reading_log']]

# LightGBM model

## Train/test split methodology

The training dataset ranges from `2016-01-01 00:00:00` to `2016-12-31 23:00:00` and the test dataset ranges from `2017-01-01 00:00:00` to `2018-12-31 23:00:00`.

So if we split the data for our own train/validation routine, we should also split the data with non-overlapping timestamps. One way to think about it is that the model should learn the day-to-day changes in energy consumption from the seasonality and weather features and not from knowing the changes in energy consumption for the same timestamps for other buildings in the same site.

In [ ]:
# We can use `sklearn.model_selection.GroupKFold` and define "groups" to be based on
# the month. Our training dataset has 12 months, and we can choose 3 splits.
kfold = GroupKFold(n_splits=3)
groups = merged_df['timestamp'].dt.month

In [ ]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

In [ ]:
# Get rid of other dataframes and save some memory before the real show begins
del train_df
del weather_train_df
del merged_df

import gc
gc.collect()

In [ ]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

In [ ]:
models = []
params = {
    "objective": "regression",
    "boosting": "gbdt",
    "num_leaves": 1280,
    "learning_rate": 0.05,
    "feature_fraction": 0.85,
    "reg_lambda": 2,
    # The actual metric we'd be measured against is RMLSE 
    # (https://www.kaggle.com/c/ashrae-energy-prediction/overview/evaluation)
    # but since we've already taken the log of the meter reading as the target
    # we should just use the RMSE metric.
    "metric": "rmse",
}
for idx, (train_index, val_index) in enumerate(kfold.split(X_df, y_df, groups)):
    print(f'Training fold {idx}')
    X_train_df, y_train_df = X_df.loc[train_index], y_df.loc[train_index]
    X_val_df, y_val_df = X_df.loc[val_index], y_df.loc[val_index]
    train_dataset = lgb.Dataset(X_train_df, label=y_train_df, 
                                categorical_feature=categorical_features)
    val_dataset = lgb.Dataset(X_val_df, label=y_val_df,
                              categorical_feature=categorical_features)
    model = lgb.train(params=params, train_set=train_dataset, num_boost_round=1000,
                      valid_sets=[val_dataset],
                      early_stopping_rounds=50, verbose_eval=25)
    models.append(model)

In [ ]:
for model in models:
    lgb.plot_importance(model)
    plt.show()

In [ ]:
print(np.mean([model.best_score['valid_0']['rmse'] for model in models]))

# Evaluating the model on the test data

In [ ]:
# Load the test dataframes first
test_df = reduce_mem_usage(load_df('test.csv'))
weather_test_df = reduce_mem_usage(load_df('weather_test.csv'))

In [ ]:
gc.collect()

In [ ]:
test_df = test_df.assign(hour_of_day=test_df['timestamp'].dt.hour, 
                         day_of_week=test_df['timestamp'].dt.dayofweek,
                         day_of_year=test_df['timestamp'].dt.dayofyear)

merged_test_df = (test_df
                  .merge(building_metadata_df, how='left', on=['building_id'])
                  .merge(weather_test_df, how='left', on=['site_id', 'timestamp']))
X_test_df = merged_test_df[feature_cols]
row_ids = merged_test_df['row_id']

In [ ]:
del test_df
del weather_test_df
del merged_test_df
gc.collect()

In [ ]:
describe_df(X_test_df)

In [ ]:
results = sum(np.expm1(model.predict(X_test_df)) / len(models) 
              for model in models)
results_df = pd.DataFrame({'row_id': row_ids, 
                           'meter_reading': np.clip(results, 0, None)})
results_df.head()

In [ ]:
results_df.to_csv('submission.csv', index=False, float_format='%.4f')

# Notes

* Why encode year_built? Isn't it already encoded?
* Play around with imputing the empty features for the weather data
* Play around with month of year vs. day of year
* Consider converting site 0 readings to kWh just like the rest
* Maybe floor count should be categorical?